In [ ]:

# todo: not working

from yahoofinancials import YahooFinancials

def get_quarter_data(financial_data, company, quarter_date):
    company_data = financial_data[company]
    for quarter in company_data:
        if quarter_date in quarter:
            return quarter[quarter_date]
    return {}

def calculate_piotroski_fscore(company, quarter_date):
    yahoo_financials = YahooFinancials(company)

    balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')
    income_statement_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'income')
    cash_statement_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'cash')

    try:
        balance_sheet_quarter_data = get_quarter_data(balance_sheet_data_qt['balanceSheetHistoryQuarterly'], company, quarter_date)
        income_statement_quarter_data = get_quarter_data(income_statement_data_qt['incomeStatementHistoryQuarterly'], company, quarter_date)
        cash_statement_quarter_data = get_quarter_data(cash_statement_data_qt['cashflowStatementHistoryQuarterly'], company, quarter_date)

        # Profitability
        roa = income_statement_quarter_data['netIncome'] / balance_sheet_quarter_data['totalAssets']  # Return on Assets
        cfo = cash_statement_quarter_data['cashFlowFromContinuingOperatingActivities'] / balance_sheet_quarter_data['totalAssets']  # Cash Flow Return
        delta_roa = roa - cash_statement_quarter_data['netIncome'] / balance_sheet_quarter_data['totalAssets']  # Change in Return of Assets
        accrual = roa - cfo  # Quality of earnings

        # Leverage, Liquidity and Source of Funds
        delta_lever = (balance_sheet_quarter_data['shortLongTermDebtTotal'] + balance_sheet_quarter_data['longTermDebtTotal']) / balance_sheet_quarter_data['totalAssets'] # Change in Leverage
        delta_liquid = balance_sheet_quarter_data['currentRatio']  # Change in Liquidity
        eq_offer = income_statement_quarter_data['commonStockIssuance']

        # Operating Efficiency
        delta_margin = income_statement_quarter_data['grossProfit'] / income_statement_quarter_data['totalRevenue']  # Change in Gross Margin
        delta_turn = income_statement_quarter_data['totalRevenue'] / balance_sheet_quarter_data['totalAssets']  # Change in Asset Turnover ratio

        # Piotroski f score calculation
        f_score = (roa > 0).astype(int) + (cfo > 0).astype(int) + (delta_roa > 0).astype(int) + (accrual < 0).astype(int) \
                  + (delta_lever < 0).astype(int) + (delta_liquid > 0).astype(int) + (eq_offer <= 0).astype(int) \
                  + (delta_margin > 0).astype(int) + (delta_turn > 0).astype(int)

        return f_score

    except KeyError as e:
        print(f"KeyError occurred: {e}")
        return None

company = 'AAPL'
quarter_date = '2023-03-31'

f_score = calculate_piotroski_fscore(company, quarter_date)

if f_score is not None:
    print(f"Piotroski f-score for {company} on {quarter_date} is {f_score}")
